All functions should be run once before to get them to compile w/ numba!!! 
Specific iterarion numbers should be taken from the full numba run so that it's accurate in relation to the others!

In [1]:
import timeit

from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from astropy import units as u
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()
timing_dict_list = {}
timing_dict_list_percall = {}
timing_dict_avg = {}
per_call_dict = {}

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
[py.warnings         ][WARNING]  /h

[tardis.simulation.base][INFO   ]  Starting iteration 7/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ] 

[tardis.simulation.base][INFO   ]  t_inner 10123.784 K -- next t_inner 11175.750 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 14/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15    

In [2]:
"""This is a dictionary of the amount of runs of each function 
based off of 1 run of trace_packet with the default r_packet"""
function_run_dict = {}
function_run_dict["trace_packet"] = 1
function_run_dict["calculate_distance_boundary"] = 1
function_run_dict["calculate_distance_electron"] = 1
function_run_dict["get_doppler_factor"] = 1
function_run_dict["get_doppler_factor_partial_relativity"] = 1
function_run_dict["calculate_distance_line"] = 1
function_run_dict["calculate_tau_electron"] = 1
function_run_dict["update_line_estimators"] = 0
#function_run_dict["test_for_close_line"] = 0

#This can be increased to scale the amount of times the stmt is called
scalar_loop = 5

#This can be increased to increase the number of calls proportionally
scalar_number = 10000
for item in function_run_dict:
    function_run_dict[item] *= scalar_number

In [3]:
import numpy as np
from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)

from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

In [4]:
#This is the contents of basic_types, which must be defined in another file so it can be imported in stmt
"""def get_numba_model(sim):
    runner = sim.runner
    model = sim.model
    return NumbaModel(
        runner.r_inner_cgs,
        runner.r_outer_cgs,
        model.time_explosion.to("s").value,
    )

def get_r_packet(**kwargs):
    r_packet_kwargs = dict(
        r = 7.5e14,
        nu = 0.4,
        mu = 0.3,
        energy = 0.9,
        seed = 1963,
        index = 0,
        is_close_line = 0,
    )
    r_packet_kwargs.update(kwargs)
    return RPacket(**r_packet_kwargs)

def get_numba_plasma(sim):
    return numba_plasma_initialize(sim.plasma, sim.runner.line_interaction_type)

def get_estimator(sim):
    runner = sim.runner
    return Estimators(
        runner.j_estimator,
        runner.nu_bar_estimator,
        runner.j_blue_estimator, 
        runner.Edotlu_estimator,
    )"""
x=5

In [5]:
from tardis.montecarlo.montecarlo_numba.r_packet import trace_packet
from tardis.montecarlo.montecarlo_numba.calculate_distances import (calculate_distance_boundary,
                                                                   calculate_distance_line,
                                                                   calculate_distance_electron)
from tardis.montecarlo.montecarlo_numba.opacities import calculate_tau_electron
from tardis.montecarlo.montecarlo_numba.estimators import update_line_estimators
from tardis.montecarlo.montecarlo_numba.frame_transformations import (get_doppler_factor,
                                                                     get_doppler_factor_partial_relativity)
from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)


In [7]:
r_packet0 = get_r_packet()
numba_model0 = get_numba_model(sim)
numba_plasma0 = get_numba_plasma(sim)
estimator0 = get_estimator(sim)
r_packet0.initialize_line_id(numba_plasma0, numba_model0)
print(r_packet0.next_line_id)
trace_packet(r_packet0, numba_model0, numba_plasma0, estimator0)
print(r_packet0.next_line_id)

29223
29223


In [8]:
#%%timeit

#trace_packet(r_packet1, numba_model1, numba_plasma1, estimator1)

In [9]:
trace_packet_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

#Function being tested
from tardis.montecarlo.montecarlo_numba.r_packet import trace_packet


r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)
"""
time_sum = 0
timing_dict_list["trace_packet"] = []
timing_dict_list_percall["trace_packet"] = []
for i in range(scalar_loop):
    trace_packet_time = timeit.timeit(setup = trace_packet_setup, stmt='trace_packet(r_packet, numba_model, numba_plasma, estimator)', number = function_run_dict["trace_packet"])
    time_sum += trace_packet_time
    timing_dict_list["trace_packet"].append(trace_packet_time)
    timing_dict_list_percall["trace_packet"].append(trace_packet_time/function_run_dict["trace_packet"])

timing_dict_avg["trace_packet"] = time_sum/scalar_loop
per_call_dict["trace_packet"] = time_sum/(scalar_loop*function_run_dict["trace_packet"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
[py.warnings         ][WARNING]  /h

[tardis.simulation.base][INFO   ]  Starting iteration 7/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ] 

[tardis.simulation.base][INFO   ]  t_inner 10123.784 K -- next t_inner 11175.750 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 14/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15    

[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    nex

[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 er

...

[tardis.simulation.base][INFO   ]  t_inner 11285.966 K -- next t_inner 10060.511 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 9/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     

[tardis.simulation.base][INFO   ]  t_inner 10152.603 K -- next t_inner 11168.529 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 16/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15    

[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10134.733717  11468.021903  0.508204  0.548671
	5      10222.890845  11499.835340  0.197026  0.221374
	10     10144.463169  11306.370330  0.117179  0.136291
	15      9873.477846  10953.828130  0.086941  0.102617

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg

[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 e

[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 e

In [10]:
timing_dict_avg["trace_packet"]

0.049291756599996

In [11]:
r_packet1 = get_r_packet()
numba_model1 = get_numba_model(sim)
numba_plasma1 = get_numba_plasma(sim)
estimator1 = get_estimator(sim)
r_packet1.initialize_line_id(numba_plasma1, numba_model1)

r_inner1 = numba_model1.r_inner[r_packet1.current_shell_id]
r_outer1 = numba_model1.r_outer[r_packet1.current_shell_id]

calculate_distance_boundary(r_packet1.r, r_packet1.mu, r_inner1, r_outer1)

(843684056256104.1, 1)

In [12]:
calculate_distance_boundary_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

#Function being tested
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_boundary

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)

r_inner = numba_model.r_inner[r_packet.current_shell_id]
r_outer = numba_model.r_outer[r_packet.current_shell_id]

"""
time_sum = 0
timing_dict_list["calculate_distance_boundary"] = []
timing_dict_list_percall["calculate_distance_boundary"] = []
for i in range(scalar_loop):
    calculate_distance_boundary_time = timeit.timeit(setup = calculate_distance_boundary_setup, stmt='calculate_distance_boundary(r_packet.r, r_packet.mu, r_inner, r_outer)', number = function_run_dict["calculate_distance_boundary"])
    time_sum += calculate_distance_boundary_time
    timing_dict_list["calculate_distance_boundary"].append(calculate_distance_boundary_time)
    timing_dict_list_percall["calculate_distance_boundary"].append(calculate_distance_boundary_time/function_run_dict["calculate_distance_boundary"])

timing_dict_avg["calculate_distance_boundary"] = time_sum/scalar_loop
per_call_dict["calculate_distance_boundary"] = time_sum/(scalar_loop*function_run_dict["calculate_distance_boundary"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10134.733717  0.400392  0.508204
	5      9852.611678  10222.890845  0.211205  0.197026
	10     9779.813302  10144.463169  0.142695  0.117179
	15     9708.082813   9873.477846  0.104556  0.086941

 (base.py:433)
[tardis.s

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     

...

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11468.021903  10497.008397  0.548671  0.442299
	5      11499.835340  10799.075913  0.221374  0.164288
	10     11306.370330  10623.433274  0.136291  0.101820
	15     1

[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10635.099217  11500.557678  0.439520  0.506895
	5      11035.731464  11715.267406  0.158320  0.196690
	10     10717.471486  11451.270656  0.101642  0.123243
	15    

[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11420.979987  10691.849845  0.499488  0.447482
	5      11711.923994  11042.045712  0.186857  0.164090
	10     11421.346118  10980.886609  0.119700  0.097369
	15    

In [13]:
timing_dict_avg["calculate_distance_boundary"]

0.0042908602001261896

In [14]:
r_packet2 = get_r_packet()
numba_model2 = get_numba_model(sim)
numba_plasma2 = get_numba_plasma(sim)
estimator2 = get_estimator(sim)
r_packet2.initialize_line_id(numba_plasma2, numba_model2)

cur_electron_density = numba_plasma2.electron_density[r_packet2.current_shell_id]
tau_event = -np.log(np.random.random())
calculate_distance_electron(cur_electron_density, tau_event)

250938323818318.44

In [15]:
calculate_distance_electron_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

#Function being tested
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_electron
import numpy as np

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)

cur_electron_density = numba_plasma.electron_density[r_packet.current_shell_id]
tau_event = -np.log(np.random.random())

"""
time_sum = 0
timing_dict_list["calculate_distance_electron"] = []
timing_dict_list_percall["calculate_distance_electron"] = []
for i in range(scalar_loop):
    calculate_distance_electron_time = timeit.timeit(setup = calculate_distance_electron_setup, stmt='calculate_distance_electron(cur_electron_density, tau_event)', number = function_run_dict["calculate_distance_electron"])
    time_sum += calculate_distance_electron_time
    timing_dict_list["calculate_distance_electron"].append(calculate_distance_electron_time)
    timing_dict_list_percall["calculate_distance_electron"].append(calculate_distance_electron_time/function_run_dict["calculate_distance_electron"])
    
timing_dict_avg["calculate_distance_electron"] = time_sum/scalar_loop
per_call_dict["calculate_distance_electron"] = time_sum/(scalar_loop*function_run_dict["calculate_distance_electron"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10134.733717  0.400392  0.508204
	5      9852.611678  10222.890845  0.211205  0.197026
	10     9779.813302  10144.463169  0.142695  0.117179
	15     9708.082813   9873.477846  0.104556  0.086941

 (base.py:433)
[tardis.s

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     

...

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11468.021903  10497.008397  0.548671  0.442299
	5      11499.835340  10799.075913  0.221374  0.164288
	10     11306.370330  10623.433274  0.136291  0.101820
	15     1

[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10635.099217  11500.557678  0.439520  0.506895
	5      11035.731464  11715.267406  0.158320  0.196690
	10     10717.471486  11451.270656  0.101642  0.123243
	15    

[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11420.979987  10691.849845  0.499488  0.447482
	5      11711.923994  11042.045712  0.186857  0.164090
	10     11421.346118  10980.886609  0.119700  0.097369
	15    

In [16]:
timing_dict_avg["calculate_distance_electron"]

0.003057585399074014

In [17]:
r_packet3 = get_r_packet()
numba_model3 = get_numba_model(sim)
numba_plasma3 = get_numba_plasma(sim)
estimator3 = get_estimator(sim)
r_packet3.initialize_line_id(numba_plasma3, numba_model3)

get_doppler_factor(r_packet3.r, r_packet3.mu, numba_model3.time_explosion)

0.9933180269391396

In [18]:
get_doppler_factor_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

#Function being tested
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)
"""
time_sum = 0
timing_dict_list["get_doppler_factor"] = []
timing_dict_list_percall["get_doppler_factor"] = []
for i in range(scalar_loop):
    get_doppler_factor_time = timeit.timeit(setup = get_doppler_factor_setup, stmt='get_doppler_factor(r_packet.r, r_packet.mu, numba_model.time_explosion)', number = function_run_dict["get_doppler_factor"])    
    time_sum += get_doppler_factor_time
    timing_dict_list["get_doppler_factor"].append(get_doppler_factor_time)
    timing_dict_list_percall["get_doppler_factor"].append(get_doppler_factor_time/function_run_dict["get_doppler_factor"])
    
timing_dict_avg["get_doppler_factor"] = time_sum/scalar_loop
per_call_dict["get_doppler_factor"] = time_sum/(scalar_loop*function_run_dict["get_doppler_factor"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10134.733717  0.400392  0.508204
	5      9852.611678  10222.890845  0.211205  0.197026
	10     9779.813302  10144.463169  0.142695  0.117179
	15     9708.082813   9873.477846  0.104556  0.086941

 (base.py:433)
[tardis.s

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     

...

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11468.021903  10497.008397  0.548671  0.442299
	5      11499.835340  10799.075913  0.221374  0.164288
	10     11306.370330  10623.433274  0.136291  0.101820
	15     1

[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10635.099217  11500.557678  0.439520  0.506895
	5      11035.731464  11715.267406  0.158320  0.196690
	10     10717.471486  11451.270656  0.101642  0.123243
	15    

[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11420.979987  10691.849845  0.499488  0.447482
	5      11711.923994  11042.045712  0.186857  0.164090
	10     11421.346118  10980.886609  0.119700  0.097369
	15    

In [19]:
timing_dict_avg["get_doppler_factor"]

0.004442828200262738

In [20]:
r_packet4 = get_r_packet()
numba_model4 = get_numba_model(sim)
numba_plasma4 = get_numba_plasma(sim)
estimator4 = get_estimator(sim)
r_packet4.initialize_line_id(numba_plasma4, numba_model4)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model4.time_explosion
beta = r_packet4.r * inv_t * inv_c

get_doppler_factor_partial_relativity(r_packet4.mu, beta)

0.9933180269391396

In [21]:
get_doppler_factor_partial_relativity_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)
from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)

#Function being tested
from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model.time_explosion
beta = r_packet.r * inv_t * inv_c
"""
time_sum = 0
timing_dict_list["get_doppler_factor_partial_relativity"] = []
timing_dict_list_percall["get_doppler_factor_partial_relativity"] = []
for i in range(scalar_loop):
    get_doppler_factor_partial_relativity_time = timeit.timeit(setup = get_doppler_factor_partial_relativity_setup, stmt='get_doppler_factor_partial_relativity(r_packet.mu, beta)', number = function_run_dict["get_doppler_factor_partial_relativity"])
    time_sum += get_doppler_factor_partial_relativity_time
    timing_dict_list["get_doppler_factor_partial_relativity"].append(get_doppler_factor_partial_relativity_time)
    timing_dict_list_percall["get_doppler_factor_partial_relativity"].append(get_doppler_factor_partial_relativity_time/function_run_dict["get_doppler_factor_partial_relativity"])
    
timing_dict_avg["get_doppler_factor_partial_relativity"] = time_sum/scalar_loop
per_call_dict["get_doppler_factor_partial_relativity"] = time_sum/(scalar_loop*function_run_dict["get_doppler_factor_partial_relativity"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10134.733717  0.400392  0.508204
	5      9852.611678  10222.890845  0.211205  0.197026
	10     9779.813302  10144.463169  0.142695  0.117179
	15     9708.082813   9873.477846  0.104556  0.086941

 (base.py:433)
[tardis.s

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     

...

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11468.021903  10497.008397  0.548671  0.442299
	5      11499.835340  10799.075913  0.221374  0.164288
	10     11306.370330  10623.433274  0.136291  0.101820
	15     1

[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10635.099217  11500.557678  0.439520  0.506895
	5      11035.731464  11715.267406  0.158320  0.196690
	10     10717.471486  11451.270656  0.101642  0.123243
	15    

[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11420.979987  10691.849845  0.499488  0.447482
	5      11711.923994  11042.045712  0.186857  0.164090
	10     11421.346118  10980.886609  0.119700  0.097369
	15    

In [22]:
timing_dict_avg["get_doppler_factor_partial_relativity"]

0.0021828318000189028

In [23]:
r_packet5 = get_r_packet()
numba_model5 = get_numba_model(sim)
numba_plasma5 = get_numba_plasma(sim)
estimator5 = get_estimator(sim)
r_packet5.initialize_line_id(numba_plasma5, numba_model5)


inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model5.time_explosion
beta = r_packet5.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet5.mu, beta)

comov_nu = r_packet5.nu * doppler_factor
is_last_line = True
cur_line_id =  r_packet5.next_line_id
nu_line = numba_plasma5.line_list_nu[cur_line_id]

calculate_distance_line(
        r_packet5,
        comov_nu,
        is_last_line,
        nu_line,
        numba_model5.time_explosion)
    

1e+99

In [24]:
calculate_distance_line_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity
from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)

#Function being tested
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model.time_explosion
beta = r_packet.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet.mu, beta)

comov_nu = r_packet.nu * doppler_factor
is_last_line = True
cur_line_id =  r_packet.next_line_id
nu_line = numba_plasma.line_list_nu[cur_line_id]

"""
time_sum = 0
timing_dict_list["calculate_distance_line"] = []
timing_dict_list_percall["calculate_distance_line"] = []
for i in range(scalar_loop):
    calculate_distance_line_time = timeit.timeit(setup = calculate_distance_line_setup, stmt='calculate_distance_line(r_packet, comov_nu, is_last_line, nu_line,numba_model.time_explosion)', number = function_run_dict["calculate_distance_line"])
    time_sum += calculate_distance_line_time
    timing_dict_list["calculate_distance_line"].append(calculate_distance_line_time)
    timing_dict_list_percall["calculate_distance_line"].append(calculate_distance_line_time/function_run_dict["calculate_distance_line"])
    
timing_dict_avg["calculate_distance_line"] = time_sum/scalar_loop
per_call_dict["calculate_distance_line"] = time_sum/(scalar_loop*function_run_dict["calculate_distance_line"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10134.733717  0.400392  0.508204
	5      9852.611678  10222.890845  0.211205  0.197026
	10     9779.813302  10144.463169  0.142695  0.117179
	15     9708.082813   9873.477846  0.104556  0.086941

 (base.py:433)
[tardis.s

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     

...

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11468.021903  10497.008397  0.548671  0.442299
	5      11499.835340  10799.075913  0.221374  0.164288
	10     11306.370330  10623.433274  0.136291  0.101820
	15     1

[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10635.099217  11500.557678  0.439520  0.506895
	5      11035.731464  11715.267406  0.158320  0.196690
	10     10717.471486  11451.270656  0.101642  0.123243
	15    

[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11420.979987  10691.849845  0.499488  0.447482
	5      11711.923994  11042.045712  0.186857  0.164090
	10     11421.346118  10980.886609  0.119700  0.097369
	15    

In [25]:
timing_dict_avg["calculate_distance_line"]

0.014668892799090828

In [26]:
r_packet6 = get_r_packet()
numba_model6 = get_numba_model(sim)
numba_plasma6 = get_numba_plasma(sim)
estimator6 = get_estimator(sim)
r_packet6.initialize_line_id(numba_plasma6, numba_model6)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model5.time_explosion
beta = r_packet6.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet6.mu, beta)

comov_nu = r_packet6.nu * doppler_factor
is_last_line = True
cur_line_id =  r_packet6.next_line_id
nu_line = numba_plasma6.line_list_nu[cur_line_id]


cur_electron_density = numba_plasma6.electron_density[r_packet6.current_shell_id]
distance_trace = calculate_distance_line(r_packet6, comov_nu, is_last_line, nu_line, numba_model6.time_explosion)
calculate_tau_electron(cur_electron_density, distance_trace)

1.9398363751857967e+84

In [27]:
calculate_tau_electron_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line
from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)

#Function being tested
from tardis.montecarlo.montecarlo_numba.opacities import calculate_tau_electron

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model.time_explosion
beta = r_packet.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet.mu, beta)

comov_nu = r_packet.nu * doppler_factor
is_last_line = True
cur_line_id =  r_packet.next_line_id
nu_line = numba_plasma.line_list_nu[cur_line_id]

cur_electron_density = numba_plasma.electron_density[r_packet.current_shell_id]
distance_trace = calculate_distance_line(r_packet, comov_nu, is_last_line, nu_line, numba_model.time_explosion)
"""
time_sum = 0
timing_dict_list["calculate_tau_electron"] = []
timing_dict_list_percall["calculate_tau_electron"] = []
for i in range(scalar_loop):
    calculate_tau_electron_time = timeit.timeit(setup = calculate_tau_electron_setup, stmt='calculate_tau_electron(cur_electron_density, distance_trace)', number = function_run_dict["calculate_tau_electron"])
    time_sum += calculate_tau_electron_time
    timing_dict_list["calculate_tau_electron"].append(calculate_tau_electron_time)
    timing_dict_list_percall["calculate_tau_electron"].append(calculate_tau_electron_time/function_run_dict["calculate_tau_electron"])

timing_dict_avg["calculate_tau_electron"] = time_sum/scalar_loop
per_call_dict["calculate_tau_electron"] = time_sum/(scalar_loop*function_run_dict["calculate_tau_electron"])

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.94275e+42 erg / s Luminosity absorbed = 2.65805e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10134.733717  0.400392  0.508204
	5      9852.611678  10222.890845  0.211205  0.197026
	10     9779.813302  10144.463169  0.142695  0.117179
	15     9708.082813   9873.477846  0.104556  0.086941

 (base.py:433)
[tardis.s

[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.33305e+43 erg / s Luminosity absorbed = 4.33011e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10569.747403  11547.678162  0.442290  0.500345
	5      11099.194259  11669.739508  0.153253  0.198338
	10     10809.397890  11493.569590  0.097256  0.121915
	15     

...

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.44283e+42 erg / s Luminosity absorbed = 2.70894e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11547.678162  10635.099217  0.500345  0.439520
	5      11669.739508  11035.731464  0.198338  0.158320
	10     11493.569590  10717.471486  0.121915  0.101642
	15     11083.769586  10404.049299  0.092834  0.075794

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.27820e+43 erg / s Luminosity absorbed = 4.15606e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10637.774893  11420.979987  0.452668  0.499488
	5      11002.647927  11711.923994  0.166197  0.186857
	10     10832.042108  11421.346118  0.101284  0.119700
	15     10350.602742  11076.434180  0.079468  0.089402

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

[tardis.simulation.base][INFO   ]  t_inner 11472.055 K -- next t_inner 9949.978 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 3/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.12624e+42 erg / s Luminosity absorbed = 2.54722e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11468.021903  10497.008397  0.548671  0.442299
	5      11499.835340  10799.075913  0.221374  0.164288
	10     11306.370330  10623.433274  0.136291  0.101820
	15     1

[tardis.simulation.base][INFO   ]  t_inner 10060.511 K -- next t_inner 11268.877 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.32925e+43 erg / s Luminosity absorbed = 4.25943e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10635.099217  11500.557678  0.439520  0.506895
	5      11035.731464  11715.267406  0.158320  0.196690
	10     10717.471486  11451.270656  0.101642  0.123243
	15    

[tardis.simulation.base][INFO   ]  t_inner 11168.529 K -- next t_inner 10167.208 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 17/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars
  (si.m, si.Hz, lambda x: _si.c.value / x),
 (warnings.py:110)
Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.81647e+42 erg / s Luminosity absorbed = 2.81681e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11420.979987  10691.849845  0.499488  0.447482
	5      11711.923994  11042.045712  0.186857  0.164090
	10     11421.346118  10980.886609  0.119700  0.097369
	15    

In [28]:
print(timing_dict_avg["calculate_tau_electron"])

0.0026907712002866902


In [29]:
r_packet7 = get_r_packet()
numba_model7 = get_numba_model(sim)
numba_plasma7 = get_numba_plasma(sim)
estimator7 = get_estimator(sim)
r_packet7.initialize_line_id(numba_plasma7, numba_model7)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model5.time_explosion
beta = r_packet7.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet7.mu, beta)

comov_nu = r_packet7.nu * doppler_factor
is_last_line = True
cur_line_id =  r_packet7.next_line_id
nu_line = numba_plasma7.line_list_nu[cur_line_id]
distance_trace = calculate_distance_line(r_packet7, comov_nu, is_last_line, nu_line, numba_model7.time_explosion)

update_line_estimators(estimator7, r_packet7, cur_line_id, distance_trace, numba_model7.time_explosion)

In [30]:
update_line_estimators_setup = """
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')
from tardis import run_tardis
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)
sim.run()

from tardis.montecarlo.montecarlo_numba.r_packet import (RPacket)
from tardis.montecarlo.montecarlo_numba.numba_interface import (NumbaModel, 
                                                                NumbaPlasma, 
                                                                numba_plasma_initialize, 
                                                                Estimators)
from docs.Profiling.basic_types import (get_numba_model,
                                       get_r_packet,
                                       get_numba_plasma,
                                       get_estimator)

from tardis.montecarlo.montecarlo_numba.frame_transformations import get_doppler_factor_partial_relativity
from tardis.montecarlo.montecarlo_numba.calculate_distances import calculate_distance_line
from tardis.montecarlo.montecarlo_numba.numba_config import (C_SPEED_OF_LIGHT, ENABLE_FULL_RELATIVITY)

#Function being tested
from tardis.montecarlo.montecarlo_numba.estimators import update_line_estimators

r_packet = get_r_packet()
numba_model = get_numba_model(sim)
numba_plasma = get_numba_plasma(sim)
estimator = get_estimator(sim)
r_packet.initialize_line_id(numba_plasma, numba_model)

inv_c = 1 / C_SPEED_OF_LIGHT
inv_t = 1 / numba_model.time_explosion
beta = r_packet.r * inv_t * inv_c

doppler_factor = get_doppler_factor_partial_relativity(r_packet.mu, beta)

comov_nu = r_packet.nu * doppler_factor
is_last_line = True
cur_line_id =  r_packet.next_line_id
nu_line = numba_plasma.line_list_nu[cur_line_id]

distance_trace = calculate_distance_line(r_packet, comov_nu, is_last_line, nu_line, numba_model.time_explosion)
"""
try:
    time_sum = 0
    timing_dict_list["update_line_estimators"] = []
    timing_dict_list_percall["update_line_estimators"] = []
    for i in range(scalar_loop):
        update_line_estimators_time = timeit.timeit(setup = update_line_estimators_setup, stmt='update_line_estimators(estimator, r_packet, cur_line_id, distance_trace, numba_model.time_explosion)', number = function_run_dict["update_line_estimators"])
        time_sum += update_line_estimators_time
        timing_dict_list["update_line_estimators"].append(update_line_estimators_time)
        timing_dict_list_percall["update_line_estimators"].append(update_line_estimators_time/function_run_dict["update_line_estimators"])

    timing_dict_avg["update_line_estimators"] = time_sum/scalar_loop
    per_call_dict["update_line_estimators"] = time_sum/(scalar_loop*function_run_dict["update_line_estimators"])
except ZeroDivisionError:
    timing_dict_list["update_line_estimators"] = []
    timing_dict_list_percall["update_line_estimators"] = []
    for i in range(scalar_loop):
        timing_dict_list["update_line_estimators"].append(0)
        timing_dict_list_percall["update_line_estimators"].append(0)
    timing_dict_avg["update_line_estimators"] = 0
    per_call_dict["update_line_estimators"] = 0

[tardis.io.atom_data.atom_web_download][INFO   ]  Downloading atomic data from https://media.githubusercontent.com/media/tardis-sn/tardis-refdata/master/atom_data/kurucz_cd23_chianti_H_He.h5 to /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (atom_web_download.py:46)
[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:91)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/kevin/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:34)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:179)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data, yg_data, two_photon_data (base.py:183)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:323)
Running post-merge numba montecarlo

Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.39001e+42 erg / s Luminosity absorbed = 2.69230e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      11567.208184  10569.747403  0.504178  0.442290
	5      11653.124121  11099.194259  0.203042  0.153253
	10     11425.955010  10809.397890  0.125249  0.097256
	15     11082.155938  10454.841299  0.094028  0.073809

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 10044.200 K -- next t_inner 11285.966 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 8/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars


Running post-merge numba montecarlo (with C close lines)!
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28354e+43 erg / s Luminosity absorbed = 4.15095e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:438)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10680.784661  11437.188130  0.439281  0.496749
	5      10887.379428  11571.106856  0.170137  0.196714
	10     10825.792025  11378.137918  0.100780  0.120954
	15     10402.120228  10957.211814  0.077634  0.092896

 (base.py:433)
[tardis.simulation.base][INFO   ]  t_inner 11175.750 K -- next t_inner 10152.603 K (base.py:434)
[tardis.simulation.base][INFO   ]  Starting iteration 15/20 (base.py:323)
[py.warnings         ][WARNING]  /home/kevin/miniconda3/envs/tardis/lib/python3.7/site-packages/astropy/units/equivalencies.py:124: RuntimeWarning: divide by zero encountered in double_scalars

In [31]:
print(timing_dict_avg["update_line_estimators"])

0


In [32]:
#So this process is somewhat fundamentally flawed, and to make it better I need to do a bit more digging. 
#As I have static packets the total calls doesn't have clear effect, as some may be bigger or smaller. 
#This sum should be less than 100%, as I am not doing test_for_close_line!

In [33]:
for item in timing_dict_list:
    print("{:<30s} {:<30} {:<30}".format(item[:30], "tottime per iter", "avg time percall"))
    for x, items in enumerate(timing_dict_list[item]):
        print("{:30s} {:<30} {:<30}".format(" ", items, timing_dict_list_percall[item][x]))
    print("{:30s}".format(" ") + "-"*60)
    print("{:30s} {:<30} {:<30}".format("averages for time and percall", timing_dict_avg[item], 
                                        per_call_dict[item]))
    print("\n")

trace_packet                   tottime per iter               avg time percall              
                               0.04979280299812672            4.979280299812672e-06         
                               0.03283064999777707            3.2830649997777073e-06        
                               0.049663682002574205           4.96636820025742e-06          
                               0.060204261000762926           6.0204261000762925e-06        
                               0.05396738700073911            5.396738700073911e-06         
                              ------------------------------------------------------------
averages for time and percall  0.049291756599996              4.929175659999601e-06         


calculate_distance_boundary    tottime per iter               avg time percall              
                               0.005901500997424591           5.901500997424591e-07         
                               0.0033458310026617255          3.345831

In [34]:
#This is w/ scalar_loop = 20, scalar_number = 10000

In [35]:
print("{:<30s} {:<30} {:<30} % of trace_packet".format("function", "avg tottime", "avg time percall"))
sum = 0
for item in timing_dict_avg:
    print("{:<30s} {:<30} {:<30} {}".format(item[:30], timing_dict_avg[item], per_call_dict[item], 
                        (float(timing_dict_avg[item]/float(timing_dict_avg["trace_packet"]))*100)))
    if (item != "trace_packet"):
        sum += timing_dict_avg[item]
print("{:<60}  {:<30s} {:<}".format(" ", "sum of subfunctions tottime", (sum/timing_dict_avg["trace_packet"])*100))

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.049291756599996              4.929175659999601e-06          100.0
calculate_distance_boundary    0.0042908602001261896          4.29086020012619e-07           8.705025943681905
calculate_distance_electron    0.003057585399074014           3.057585399074014e-07          6.203035984062012
get_doppler_factor             0.004442828200262738           4.442828200262738e-07          9.013329016282407
get_doppler_factor_partial_rel 0.0021828318000189028          2.1828318000189027e-07         4.428391176505728
calculate_distance_line        0.014668892799090828           1.4668892799090826e-06         29.759322472780404
calculate_tau_electron         0.0026907712002866902          2.69077120028669e-07           5.458866524320435
update_line_estimators         0                              0                              0.0
                          

In [36]:
print("weird result w/ tardis_example, scalar_loop = 5, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04725032119968091            4.7250321199680915e-06         100.0
calculate_distance_boundary    0.005027625799993984           5.027625799993985e-07          10.640405551418636
calculate_distance_electron    0.002631970999573241           2.631970999573241e-07          5.570271127788684
get_doppler_factor             0.005451651599651086           5.451651599651086e-07          11.537808550786956
get_doppler_factor_partial_rel 0.002278191399818752           2.278191399818752e-07          4.821536323935375
calculate_distance_line        0.010987908599781803           1.0987908599781803e-06         23.254674932994966
calculate_tau_electron         0.001848176600469742           1.848176600469742e-07          3.9114582791073667
update_line_estimators         0                              0                              0.0
                                                              sum of subfunctions tottime    59.73615476603199
""")

weird result w/ tardis_example, scalar_loop = 5, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04725032119968091            4.7250321199680915e-06         100.0
calculate_distance_boundary    0.005027625799993984           5.027625799993985e-07          10.640405551418636
calculate_distance_electron    0.002631970999573241           2.631970999573241e-07          5.570271127788684
get_doppler_factor             0.005451651599651086           5.451651599651086e-07          11.537808550786956
get_doppler_factor_partial_rel 0.002278191399818752           2.278191399818752e-07          4.821536323935375
calculate_distance_line        0.010987908599781803           1.0987908599781803e-06         23.254674932994966
calculate_tau_electron         0.001848176600469742           1.848176600469742e-07          3.9114582791073667
update_line_estimators         0                

In [37]:
print("fist success, nothing in stmt, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.05180641805027335            5.180641805027335e-06          100.0
calculate_distance_boundary    0.00800151534986071            8.00151534986071e-07           15.44502718195259
calculate_distance_electron    0.004804575649995968           4.804575649995968e-07          9.274093501954855
get_doppler_factor             0.0062014484000428634          6.201448400042863e-07          11.97042496554178
get_doppler_factor_partial_rel 0.003323126699979184           3.323126699979184e-07          6.414507748353489
calculate_distance_line        0.012704911699802323           1.2704911699802325e-06         24.523818047936413
calculate_tau_electron         0.0036553111000102944          3.6553111000102943e-07         7.0557109284474215
update_line_estimators         0                              0                              0.0
                                                              sum of subfunctions tottime    74.68358237418654
""")

fist success, nothing in stmt, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.05180641805027335            5.180641805027335e-06          100.0
calculate_distance_boundary    0.00800151534986071            8.00151534986071e-07           15.44502718195259
calculate_distance_electron    0.004804575649995968           4.804575649995968e-07          9.274093501954855
get_doppler_factor             0.0062014484000428634          6.201448400042863e-07          11.97042496554178
get_doppler_factor_partial_rel 0.003323126699979184           3.323126699979184e-07          6.414507748353489
calculate_distance_line        0.012704911699802323           1.2704911699802325e-06         24.523818047936413
calculate_tau_electron         0.0036553111000102944          3.6553111000102943e-07         7.0557109284474215
update_line_estimators         0                  

In [38]:
print("Very close, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.049611711900161025           4.961171190016103e-06          100.0
calculate_distance_boundary    0.009409913950003101           9.409913950003101e-07          18.967122055654283
calculate_distance_electron    0.006357851049779128           6.357851049779129e-07          12.815222064043496
get_doppler_factor             0.009268622899799083           9.268622899799084e-07          18.682328314836884
get_doppler_factor_partial_rel 0.003563347049839649           3.563347049839649e-07          7.182471463614388
calculate_distance_line        0.017110374550247796           1.7110374550247798e-06         34.48857919815555
calculate_tau_electron         0.0038214105999031744          3.8214105999031745e-07         7.7026380536785535
update_line_estimators         0                              0                              0.0
                                                              sum of subfunctions tottime    99.83836114998314
""")

Very close, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.049611711900161025           4.961171190016103e-06          100.0
calculate_distance_boundary    0.009409913950003101           9.409913950003101e-07          18.967122055654283
calculate_distance_electron    0.006357851049779128           6.357851049779129e-07          12.815222064043496
get_doppler_factor             0.009268622899799083           9.268622899799084e-07          18.682328314836884
get_doppler_factor_partial_rel 0.003563347049839649           3.563347049839649e-07          7.182471463614388
calculate_distance_line        0.017110374550247796           1.7110374550247798e-06         34.48857919815555
calculate_tau_electron         0.0038214105999031744          3.8214105999031745e-07         7.7026380536785535
update_line_estimators         0                              0    

In [39]:
print("Very close, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.051361916849964474           5.136191684996447e-06          100.0
calculate_distance_boundary    0.007689172699974734           7.689172699974733e-07          14.970571917002065
calculate_distance_electron    0.0038574883501496515          3.8574883501496515e-07         7.510405737811398
get_doppler_factor             0.0053859479999118776          5.385947999911878e-07          10.486267511481326
get_doppler_factor_partial_rel 0.003476134700122202           3.476134700122202e-07          6.767922447825478
calculate_distance_line        0.015309354699911636           1.5309354699911637e-06         29.80682116018461
calculate_tau_electron         0.004660611650251667           4.660611650251667e-07          9.074060969854342
update_line_estimators         0                              0                              0.0
                                                              sum of subfunctions tottime    78.61604974415921
""")

Very close, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.051361916849964474           5.136191684996447e-06          100.0
calculate_distance_boundary    0.007689172699974734           7.689172699974733e-07          14.970571917002065
calculate_distance_electron    0.0038574883501496515          3.8574883501496515e-07         7.510405737811398
get_doppler_factor             0.0053859479999118776          5.385947999911878e-07          10.486267511481326
get_doppler_factor_partial_rel 0.003476134700122202           3.476134700122202e-07          6.767922447825478
calculate_distance_line        0.015309354699911636           1.5309354699911637e-06         29.80682116018461
calculate_tau_electron         0.004660611650251667           4.660611650251667e-07          9.074060969854342
update_line_estimators         0                              0      

In [40]:
print("bit over, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04299541624986887            4.299541624986887e-06          100.0
calculate_distance_boundary    0.007351055949857255           7.351055949857255e-07          17.097301505668465
calculate_distance_electron    0.006282690300031391           6.282690300031391e-07          14.61246534635085
get_doppler_factor             0.007747141999971063           7.747141999971063e-07          18.018530056665497
get_doppler_factor_partial_rel 0.004546488550022332           4.546488550022332e-07          10.574356400227195
calculate_distance_line        0.01475706765004361            1.475706765004361e-06          34.32242070708785
calculate_tau_electron         0.005234073999781685           5.234073999781686e-07          12.173562803448027
update_line_estimators         0                              0                              0.0
                                                              sum of subfunctions tottime    106.7986368194479
""")

bit over, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04299541624986887            4.299541624986887e-06          100.0
calculate_distance_boundary    0.007351055949857255           7.351055949857255e-07          17.097301505668465
calculate_distance_electron    0.006282690300031391           6.282690300031391e-07          14.61246534635085
get_doppler_factor             0.007747141999971063           7.747141999971063e-07          18.018530056665497
get_doppler_factor_partial_rel 0.004546488550022332           4.546488550022332e-07          10.574356400227195
calculate_distance_line        0.01475706765004361            1.475706765004361e-06          34.32242070708785
calculate_tau_electron         0.005234073999781685           5.234073999781686e-07          12.173562803448027
update_line_estimators         0                              0      

In [41]:
print("extremely close, scalar_loop = 20, scalar_number = 10000")
print("""
function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04344090969989338            4.344090969989338e-06          100.0
calculate_distance_boundary    0.007661852999990515           7.661852999990515e-07          17.63741379478828
calculate_distance_electron    0.006289332049982476           6.289332049982477e-07          14.477901345601685
get_doppler_factor             0.007295510800213378           7.295510800213379e-07          16.794102265844778
get_doppler_factor_partial_rel 0.0050076760999218095          5.00767609992181e-07           11.527558088715855
calculate_distance_line        0.01249697484936405            1.2496974849364051e-06         28.767755868139023
calculate_tau_electron         0.005331157749878912           5.331157749878912e-07          12.272205593088666
update_line_estimators         0                              0                              0.0
                                                              sum of subfunctions tottime    101.47693695617828
""")

extremely close, scalar_loop = 20, scalar_number = 10000

function                       avg tottime                    avg time percall               % of trace_packet
trace_packet                   0.04344090969989338            4.344090969989338e-06          100.0
calculate_distance_boundary    0.007661852999990515           7.661852999990515e-07          17.63741379478828
calculate_distance_electron    0.006289332049982476           6.289332049982477e-07          14.477901345601685
get_doppler_factor             0.007295510800213378           7.295510800213379e-07          16.794102265844778
get_doppler_factor_partial_rel 0.0050076760999218095          5.00767609992181e-07           11.527558088715855
calculate_distance_line        0.01249697484936405            1.2496974849364051e-06         28.767755868139023
calculate_tau_electron         0.005331157749878912           5.331157749878912e-07          12.272205593088666
update_line_estimators         0                             